# Visão Geral: Filtragem Colaborativa Item-Item com LSH

Este notebook demonstra como construir um sistema de recomendação escalável de filtragem colaborativa item-item utilizando **Locality Sensitive Hashing (LSH)** com PySpark. O objetivo principal é encontrar de forma eficiente itens (filmes) similares com base nas avaliações dos usuários, mesmo em conjuntos de dados muito grandes.

## Abordagem

1. **Preparação dos Dados**: Carregar o dataset MovieLens e dividir em conjuntos de treino e teste.
2. **Vetorização dos Itens**: Representar cada item (filme) como um vetor esparso, onde cada dimensão corresponde a um usuário e o valor é a nota atribuída por esse usuário.
3. **Normalização**: Normalizar os vetores dos itens utilizando a norma L2, permitindo que a distância Euclidiana aproxime a similaridade cosseno.
4. **LSH para Busca de Similaridade**: Utilizar o BucketedRandomProjectionLSH para encontrar rapidamente pares de itens que provavelmente são similares, sem calcular todas as similaridades possíveis.
5. **Cálculo da Similaridade**: Para cada par de itens encontrado pelo LSH, calcular a similaridade cosseno usando a fórmula:
   
   $$
   \cos(\theta) = 1 - \frac{d^2}{2}
   $$
   onde $d$ é a distância Euclidiana entre os vetores normalizados.

6. **Predição**: Para cada usuário e filme alvo no conjunto de teste, prever a nota utilizando a média ponderada das notas do usuário para filmes similares, ponderadas pela similaridade:
   
   $$
   \hat{r}_{ui} = \frac{\sum_{j \in N(i)} s_{ij} \cdot r_{uj}}{\sum_{j \in N(i)} |s_{ij}|}
   $$
   - $N(i)$: conjunto de vizinhos do filme $i$
   - $s_{ij}$: similaridade entre o filme $i$ e o filme $j$
   - $r_{uj}$: nota dada pelo usuário $u$ ao filme $j$

7. **Avaliação**: Comparar as notas previstas com as reais utilizando RMSE (Root Mean Squared Error) e MAE (Mean Absolute Error).

## Por que LSH?

A filtragem colaborativa item-item tradicional exige comparar todos os pares de itens, o que é computacionalmente caro em grandes conjuntos de dados. O LSH permite encontrar de forma eficiente apenas os pares mais promissores, tornando a abordagem escalável.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, struct, sum as sql_sum, udf, lit
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.ml.feature import Normalizer, BucketedRandomProjectionLSH
from pyspark.ml.evaluation import RegressionEvaluator
import os
import time
import pandas as pd

In [ ]:
start_time = time.time() # para controlor o tempo de duracao

In [ ]:
spark = SparkSession.builder \
    .appName("ItemItemCF") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set logging level
spark.sparkContext.setLogLevel("WARN")

25/05/30 23:57:49 WARN Utils: Your hostname, cristianonicolau.local resolves to a loopback address: 127.0.0.1; using 192.168.1.122 instead (on interface en0)
25/05/30 23:57:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/30 23:57:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/30 23:57:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
df ={
    "small": "./data/ml-latest-small/ratings.csv",
    "ml-1m": "./data/ml-1m/ratings.csv",
    "ml-10m": "./data/ml-10m/ratings.csv",
    "ml-20m": "./data/ml-20m/ratings.csv",
    "ml-25m": "./data/ml-25m/ratings.csv",
}

data_sel = "small" 

Abaixo defino os caminhos para diferentes datasets do MovieLens. A ideia é quando na primeira exucaçao para dar download aos datasets usar o codigo abaixo

In [ ]:
# dataset_links = {
#     "small": "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip",
#     "ml-1m": "https://files.grouplens.org/datasets/movielens/ml-1m.zip",
#     "ml-10m": "https://files.grouplens.org/datasets/movielens/ml-10m.zip",
#     "ml-20m": "https://files.grouplens.org/datasets/movielens/ml-20m.zip",
#     "ml-25m": "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
# }

# def download_and_extract_dataset(dataset_name):
#     if dataset_name not in dataset_links:
#         raise ValueError(f"Dataset '{dataset_name}' not found. Available datasets: {list(dataset_links.keys())}")

#     url = dataset_links[dataset_name]
#     os.system(f"wget {url} -O {dataset_name}.zip")
#     os.system(f"unzip {dataset_name}.zip -d {dataset_name}")
#     print(f"Downloaded and extracted {dataset_name} dataset.")

In [ ]:
PATH = df[data_sel]

# download_and_extract_dataset("ml-25m")  # Change to desired dataset
# PATH = "ml-25m/ratings.csv"  # Adjust path based on the dataset
data = spark.read.csv(PATH, header=True, inferSchema=True) \
            .select("userId", "movieId", "rating")

print("=== DATASET STATISTICS ===")
print("Number of ratings: ", data.count())
print("Average rating: ", data.agg(F.avg("rating")).first()[0])
print("Minimum rating: ", data.agg(F.min("rating")).first()[0])
print("Maximum rating: ", data.agg(F.max("rating")).first()[0])
print("Number of users: ", data.select("userId").distinct().count())
print("Number of movies: ", data.select("movieId").distinct().count())

data.take(5)

=== DATASET STATISTICS ===
Number of ratings:  100836
Average rating:  3.501556983616962
Minimum rating:  0.5
Maximum rating:  5.0
Number of users:  610
Number of movies:  9724


[Row(userId=1, movieId=1, rating=4.0),
 Row(userId=1, movieId=3, rating=4.0),
 Row(userId=1, movieId=6, rating=4.0),
 Row(userId=1, movieId=47, rating=5.0),
 Row(userId=1, movieId=50, rating=5.0)]

In [ ]:
# Split data into training and test sets
ratings, test = data.randomSplit([0.9, 0.1], seed=42)
print(f"Training set size: {ratings.count()}")
print(f"Test set size: {test.count()}")
ratings.cache()
test.cache()

Training set size: 90673
Test set size: 10163


DataFrame[userId: int, movieId: int, rating: double]

### Criação dos Vetores dos Itens
A eguir transformo as avaliações dos users em vetores esparsos, onde cada dimensão representa um user e o valor é a nota atribuída ao item, é necessartio para calcular similaridades entre os itens de forma eficiente.

In [ ]:
def to_sparse_vector(user_ratings, size):
    # Sort by userId to ensure strictly increasing indices
    sorted_pairs = sorted(user_ratings, key=lambda x: x.userId)
    indices = [x.userId - 1 for x in sorted_pairs] # Assuming userIds start at 1
    values = [float(x.rating) for x in sorted_pairs]
    return Vectors.sparse(size, indices, values)

In [ ]:
spark.udf.register("to_sparse_vector", to_sparse_vector)

item_user = ratings.groupBy("movieId") \
    .agg(collect_list(struct("userId", "rating")).alias("user_ratings"))

### Agrupamento das Avaliações por Item
Registramos a função de conversão para vetor esparso como UDF no Spark e agrupamos as avaliações por `movieId`, preparando os dados para a criação dos vetores de características dos itens.

### Criação dos Vetores de Características dos Itens
Para cada item, criamos um vetor esparso representando as avaliações dos users. Esses vetores serão usados para calcular similaridades entre itens e para o agrupamento via clustering.

In [ ]:
num_users = ratings.select("userId").distinct().count()

item_vectors_rdd = item_user.rdd.map(
    lambda row: Row(
        movieId=row["movieId"],
        features=to_sparse_vector(row["user_ratings"], num_users)
    )
)
item_vectors = spark.createDataFrame(item_vectors_rdd)
item_vectors.cache()
print("=== ITEM VECTORS ===")
item_vectors.show(5, truncate=False)

=== ITEM VECTORS ===


+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Normalização dos Vetores
Normalizamos os vetores dos itens utilizando a norma L2. Isso permite que a distância Euclidiana aproxime a similaridade cosseno, facilitando o uso de clustering e cálculo de similaridades.

In [ ]:
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
normalized_item_vectors = normalizer.transform(item_vectors)
normalized_item_vectors.cache() # Cache for LSH

print("Normalized Item Vectors (sample):")
normalized_item_vectors.show(5, truncate=False)

Normalized Item Vectors (sample):
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 3.2 Normalização dos Vetores

Normalizamos os vetores dos itens utilizando a norma L2. Isso permite que a distância Euclidiana aproxime a similaridade cosseno, facilitando o uso do LSH para encontrar itens similares.

## 3.3 Apply LSH

We fit an LSH model. The `bucketLength` and `numHashTables` are key parameters:
* `bucketLength`: Affects the width of the buckets. Smaller values lead to more, smaller buckets (higher precision, lower recall).
* `numHashTables`: Increases the chance of finding neighbors by using multiple hash functions (higher recall, more computation).

These parameters often require tuning based on the dataset and desired trade-off.


In [ ]:
bucket_length = 1.5
num_hash_tables = 8


lsh = BucketedRandomProjectionLSH(
    inputCol="norm_features",
    outputCol="hashes",
    bucketLength= bucket_length,
    numHashTables= num_hash_tables
)

lsh_model = lsh.fit(normalized_item_vectors)

25/05/30 23:57:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


### Configuração e Treinamento do LSH

Definimos os parâmetros do LSH (comprimento do bucket e número de tabelas de hash) e treinamos o modelo para agrupar itens similares em buckets, acelerando a busca por vizinhos próximos.

### Busca por Itens Similares

Utilizamos o método `approxSimilarityJoin` do LSH para encontrar pares de itens com distância Euclidiana abaixo de um limiar. Convertendo essa distância para similaridade cosseno, obtemos os pares de itens mais similares para recomendações.



In [ ]:
#  set um valor de threshold para a similaridade
similarity_threshold = 1.0

neighbors = lsh_model.approxSimilarityJoin(
    normalized_item_vectors,
    normalized_item_vectors,
    threshold=similarity_threshold, 
    distCol="distance"
).filter(col("datasetA.movieId") != col("datasetB.movieId")) # Exclude self-pairs

# calculamos a similaridade do cosseno
neighbors_cosine = neighbors.withColumn(
    "cosine_sim",
    1 - (col("distance") ** 2) / 2
).select(
    col("datasetA.movieId").alias("i_mv"),
    col("datasetB.movieId").alias("j_mv"),
    "cosine_sim"
)

# fazemos a uniao dos pares de itens similares
similarities = neighbors_cosine.union(
    neighbors_cosine.selectExpr("j_mv as i_mv", "i_mv as j_mv", "cosine_sim")
)

similarities.cache() # Cache for predictions

print("Similar Item Pairs (sample):")
similarities.show(10, truncate=False)

Similar Item Pairs (sample):


+----+-----+------------------+
|i_mv|j_mv |cosine_sim        |
+----+-----+------------------+
|12  |39400|0.5139088171029906|
|27  |191  |0.5587497991102406|
|30  |3475 |0.5437272478270369|
|30  |1176 |0.5970814340265322|
|30  |649  |0.6963106238227913|
|38  |78034|0.6666666666666667|
|38  |4409 |0.506171068243531 |
|43  |8809 |0.5254364539073235|
|77  |2902 |0.6396021490668313|
|92  |25870|0.5132649025747364|
+----+-----+------------------+
only showing top 10 rows



In [ ]:
# Damos join entre os pares de itens similares e o conjunto de teste
test_neighbors = test.alias("t") \
    .join(similarities.alias("s"), col("t.movieId") == col("s.i_mv"))

# Selecionamos as colunas relevantes para o teste
test_with_ratings = test_neighbors \
    .join(ratings.alias("r"), (col("t.userId") == col("r.userId")) & (col("s.j_mv") == col("r.movieId"))) \
    .select(
        col("t.userId"),
        col("t.movieId").alias("target_movie"),
        col("s.j_mv").alias("neighbor_movie"),
        col("s.cosine_sim"),
        col("r.rating").alias("neighbor_rating")
    )

print("Test Data with Neighbor Ratings (sample):")
test_with_ratings.show(5, truncate=False)

Test Data with Neighbor Ratings (sample):
+------+------------+--------------+------------------+---------------+
|userId|target_movie|neighbor_movie|cosine_sim        |neighbor_rating|
+------+------------+--------------+------------------+---------------+
|525   |260         |1210          |0.6824355345941335|4.0            |
|477   |260         |1210          |0.6824355345941335|4.5            |
|453   |260         |1210          |0.6824355345941335|3.0            |
|380   |260         |1210          |0.6824355345941335|5.0            |
|337   |260         |1210          |0.6824355345941335|5.0            |
+------+------------+--------------+------------------+---------------+
only showing top 5 rows



## Predição das Avaliações

Para prever a nota que um user daria a um filme, procuramos os vizinhos do item alvo e as avaliações do usuário nesses vizinhos. 

In [ ]:
# Calculamos a soma ponderada das classificações dos vizinhos e a soma das similaridades
weighted_sums = test_with_ratings.groupBy("userId", "target_movie").agg(
    sql_sum(col("cosine_sim") * col("neighbor_rating")).alias("weighted_rating_sum"),
    sql_sum(F.abs(col("cosine_sim"))).alias("similarity_sum") # Use absolute for the denominator
)

# Calculamos a previsão da classificação como a soma ponderada das classificações dos vizinhos dividida pela soma das similaridades
predictions = weighted_sums.withColumn(
    "pred_rating",
    F.when(
        col("similarity_sum") > 0,
        col("weighted_rating_sum") / col("similarity_sum")
    ).otherwise(None) 
).filter(col("pred_rating").isNotNull()) \
 .select("userId", "target_movie", "pred_rating")

print("Predicted Ratings (sample):")
predictions.show(5, truncate=False)


Predicted Ratings (sample):


+------+------------+------------------+
|userId|target_movie|pred_rating       |
+------+------------+------------------+
|480   |3793        |3.9043522855700528|
|599   |7193        |2.2990280109681414|
|347   |480         |3.6339209916483215|
|593   |4306        |3.0803749143352657|
|599   |6539        |3.2462228492111707|
+------+------------+------------------+
only showing top 5 rows



In [ ]:
# fazemos o join entre as previsoes e o conjunto de teste para obter as classificacoes reais
final_results = predictions.alias("p") \
    .join(test.alias("t"), (col("p.userId") == col("t.userId")) & (col("p.target_movie") == col("t.movieId"))) \
    .select(
        col("p.userId"),
        col("p.target_movie"),
        col("p.pred_rating"),
        col("t.rating").alias("actual_rating")
    )

# garantimos que as previsoes nao sejam nulas
final_results_filtered = final_results.filter(col("pred_rating").isNotNull())
final_results_filtered.cache()

print("Final Results with Predictions and Actuals (sample):")
final_results_filtered.show(10, truncate=False)


Final Results with Predictions and Actuals (sample):


+------+------------+------------------+-------------+
|userId|target_movie|pred_rating       |actual_rating|
+------+------------+------------------+-------------+
|599   |6811        |1.961901830838415 |3.0          |
|95    |6934        |4.0               |4.0          |
|574   |296         |4.646079373154682 |3.0          |
|452   |2683        |5.0               |5.0          |
|298   |296         |3.843319662443349 |4.5          |
|387   |1527        |3.394992861102659 |3.5          |
|219   |733         |4.5               |3.5          |
|307   |2           |3.0               |2.5          |
|610   |76251       |3.2955652596543876|3.5          |
|489   |1035        |4.5               |3.5          |
+------+------------+------------------+-------------+
only showing top 10 rows



## Avaliação das Predições

Comparamos as predições com as avaliações reais do conjunto de teste utilizando as métricas RMSE (Root Mean Squared Error) e MAE (Mean Absolute Error), que quantificam o erro das recomendações.

In [ ]:
# calculate RMSE
evaluator = RegressionEvaluator(
    labelCol="actual_rating",
    predictionCol="pred_rating",
    metricName="rmse"
)
rmse = evaluator.evaluate(final_results_filtered)
print(f"Root Mean Squared Error (RMSE): {rmse}")

#calculate MAE
mae_evaluator = RegressionEvaluator(
    labelCol="actual_rating",
    predictionCol="pred_rating",
    metricName="mae"
)
mae = mae_evaluator.evaluate(final_results_filtered)
print(f"Mean Absolute Error (MAE): {mae}")

Root Mean Squared Error (RMSE): 0.9127885306396452
Mean Absolute Error (MAE): 0.6628536445930843


In [ ]:
# Tempo total de execução
end_time = time.time()
execution_time = round(end_time - start_time, 2)

summary = pd.DataFrame([{
    'Dataset':  data_sel,
    'Train Size': ratings.count(),
    'Test Size': test.count(),
    'Similarity': 'Cosine (approximated with LSH)',
    'Similarity Threshold': similarity_threshold,
    'Num Hash Tables': num_hash_tables,
    'Bucket Length': bucket_length,
    'RMSE': rmse,
    'MAE': mae,
    'Execution Time (s)': execution_time
}])

# Salvar o resumo
output_path = './output/item_item_cf_summary.csv'
if os.path.exists(output_path):
    summary.to_csv(output_path, mode='a', header=False, index=False)
else:
    summary.to_csv(output_path, index=False)

print("Resultados salvos em './output/'")


Resultados salvos em './output/'


In [ ]:
spark.stop()